


# Assignment 5 on Natural Language Processing

## Date : 3rd Nov, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

### Name : Udit Desai

### Roll number : 17CS30044

The central idea of this assignment is to explore various language models specifically LSTM based and transformer. We will explore how the size of the model effects the sequence generated. We will see both character based and word based models.



Please submit with outputs. Submissions without predicted outputs will be penalized.

# Word Based LSTM model

In [37]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras

Do basic pre processing which includes lowering etc
Check the dataset and apply suitable preprocessing.

In [38]:
from google.colab import files
files.upload()

Saving Assignment_5_NLP.ipynb to Assignment_5_NLP.ipynb


{'Assignment_5_NLP.ipynb': b'{\n  "nbformat": 4,\n  "nbformat_minor": 0,\n  "metadata": {\n    "colab": {\n      "name": "Assignment-5 NLP",\n      "provenance": [],\n      "collapsed_sections": []\n    },\n    "kernelspec": {\n      "name": "python3",\n      "display_name": "Python 3"\n    }\n  },\n  "cells": [\n    {\n      "cell_type": "markdown",\n      "metadata": {\n        "id": "zyJ25uz0kSaw"\n      },\n      "source": [\n        "\\n",\n        "\\n",\n        "\\n",\n        "# Assignment 5 on Natural Language Processing\\n",\n        "\\n",\n        "## Date : 3rd Nov, 2020\\n",\n        "\\n",\n        "### Instructor : Prof. Sudeshna Sarkar\\n",\n        "\\n",\n        "### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu"\n      ]\n    },\n    {\n      "cell_type": "markdown",\n      "metadata": {\n        "id": "Ao1nhg9RknmF"\n      },\n      "source": [\n        "The central idea of this assignment is to explore various language models s

In [39]:
# Load the data and preprocess data and store corpus in raw_text

def pre_process(text) :
  ret = ""
  for i in range(len(text)):
    s = text[i]
    s = s.lower()
    t = ""
    if (s.isalpha() == False and s!='.'):
        t += ' '
    else:
        t += s 
    ret += t

    

  return ret

data = open('corpus.txt','r')
text = data.read()
raw_text = pre_process(text)
'''print(raw_text)
print("--------")
print([raw_text])
n1 = len(raw_text)
n2 = len([raw_text])
print(n1,n2)
'''

raw_text = pre_process(raw_text)
print(raw_text)

chapter i. down the rabbit hole alice was beginning to get very tired of sitting by her sister on the bank  and of having nothing to do  once or twice she had peeped into the book her sister was reading  but it had no pictures or conversations in it   and what is the use of a book   thought alice  without pictures or conversations   so she was considering in her own mind  as well as she could  for the hot day made her feel very sleepy and stupid   whether the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies  when suddenly a white rabbit with pink eyes ran close by her. there was nothing so  very  remarkable in that  nor did alice think it so  very  much out of the way to hear the rabbit say to itself   oh dear  oh dear  i shall be late    when she thought it over afterwards  it occurred to her that she ought to have wondered at this  but at the time it all seemed quite natural   but when the rabbit actually  took a watch out of its waist

In [40]:
# Hyperparameters of the model
vocab_size = 2571 # choose based on statistics
oov_tok = '<OOV>'
embedding_dim = 100
padding_type='post'
trunc_type='post'

In [41]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts([raw_text])
word_index = tokenizer.word_index

In [42]:
seq_length = 50
tokens = tokenizer.texts_to_sequences([raw_text])[0]

In [43]:
dataX = []
dataY = []

for i in range(0, len(tokens) - seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is OOV
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)
 
N = len(dataX)
print ("Total training data size: ", N)

Total training data size:  27284


In [44]:
X = numpy.array(dataX)

# one hot encode the output variable
y = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [45]:
# with embedding
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(vocab_size, activation='softmax')
])
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 100)           257100    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               84480     
_________________________________________________________________
dense_3 (Dense)              (None, 2571)              331659    
Total params: 673,239
Trainable params: 673,239
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Use validation split of 0.2 while training
model.fit(X, y, epochs=25, batch_size=128, validation_split=0.2 ) 

Epoch 1/25
171/171 [==============================] - 3s 20ms/step - loss: 6.3458 - accuracy: 0.0530 - val_loss: 6.1990 - val_accuracy: 0.0823
Epoch 2/25
171/171 [==============================] - 3s 16ms/step - loss: 5.8830 - accuracy: 0.0545 - val_loss: 6.1359 - val_accuracy: 0.0786
Epoch 3/25
171/171 [==============================] - 3s 16ms/step - loss: 5.7595 - accuracy: 0.0572 - val_loss: 6.1244 - val_accuracy: 0.0847
Epoch 4/25
171/171 [==============================] - 3s 16ms/step - loss: 5.6557 - accuracy: 0.0639 - val_loss: 6.0965 - val_accuracy: 0.0902
Epoch 5/25
171/171 [==============================] - 3s 16ms/step - loss: 5.5501 - accuracy: 0.0680 - val_loss: 6.0770 - val_accuracy: 0.0902
Epoch 6/25
171/171 [==============================] - 3s 16ms/step - loss: 5.4362 - accuracy: 0.0810 - val_loss: 6.0434 - val_accuracy: 0.0993
Epoch 7/25
171/171 [==============================] - 3s 16ms/step - loss: 5.3021 - accuracy: 0.1013 - val_loss: 5.9745 - val_accuracy: 0.1136

In [47]:
#print(tokenizer.word_index)
original = tokenizer.word_index
reverse_word_map = {}
for key in original :
  index = original[key]
  reverse_word_map[index] = key

print(original)
print(reverse_word_map)

{'<OOV>': 1, 'the': 2, 'and': 3, 'to': 4, 'a': 5, 'it': 6, 'she': 7, 'i': 8, 'of': 9, 'said': 10, 'you': 11, 'alice': 12, 'in': 13, 'was': 14, 'that': 15, 'as': 16, 'her': 17, 't': 18, 'at': 19, 's': 20, 'on': 21, 'all': 22, 'with': 23, 'had': 24, 'but': 25, 'for': 26, 'so': 27, 'they': 28, 'be': 29, 'not': 30, 'very': 31, 'what': 32, 'this': 33, 'little': 34, 'he': 35, 'out': 36, 'is': 37, 'one': 38, 'down': 39, 'up': 40, 'there': 41, 'if': 42, 'his': 43, 'then': 44, 'about': 45, 'no': 46, 'them': 47, 'know': 48, 'like': 49, 'were': 50, 'would': 51, 'went': 52, 'again': 53, 'herself': 54, 'do': 55, 'have': 56, 'when': 57, 'or': 58, 'could': 59, 'queen': 60, 'thought': 61, 'off': 62, 'time': 63, 'how': 64, 'me': 65, 'into': 66, 'see': 67, 'well': 68, 'did': 69, 'm': 70, 'who': 71, 'can': 72, 'king': 73, 'your': 74, 'don': 75, 'now': 76, 'turtle': 77, 'by': 78, 'began': 79, 'my': 80, 'its': 81, 'll': 82, 'an': 83, 'way': 84, 'hatter': 85, 'mock': 86, 'quite': 87, 'gryphon': 88, 'are': 8

In [48]:
# Complete the code to return next n words greedily
def next_tokens(input_str, n): 
	print ("Seed: \n",  input_str)
	inp = tokenizer.texts_to_sequences([input_str])[0][0:50]
	final_string = ""
# get next word index. Use reverse_word_map to get the word
	for i in range(n):
		prediction = model.predict( [inp] , verbose=0)
		ind = -1
		mx = 0
		cols = prediction.shape[1]
		
		for i in range(cols):
			mx = max(mx,prediction[0][i])
	 
		for i in range(cols):
			if (prediction[0][i]==mx) :
	 			ind = i

		
		next_word = reverse_word_map[ind]
		inp = inp[1:50] + [ind]
		final_string = final_string + " " + next_word

	return final_string

In [49]:
# pick a random seed
import random
random.seed(94)
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ' '.join([reverse_word_map[value] for value in pattern])

print(next_tokens( input_str , 10))

Seed: 
 of the table but it was too slippery and when she had tired herself out with trying the poor little thing sat down and cried come there s no use in crying like that said alice to herself rather sharply i advise you to leave off this minute she generally
 a little way that s the house said alice i


In [50]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Use first 50 tokens from given input_str as input.(Use tokenizer to split to take first 50)
print(next_tokens( pre_process(input_str) , 10))

Seed: 
 the boy laughed at the fright he had caused. this time  the villagers left angrily. the third day  as the boy went up the small hill  he suddenly saw a wolf attacking his sheep. he cried as hard as he could   wolf  wolf  wolf    but not  a single villager came to help him. the villagers thought that he was trying to fool them again and did not come to rescue  him or his sheep.
 that said the mock turtle i m not a little


# Character based LSTM Model 1

In [51]:
# User the preprocess data and create raw_text
#print(text)

def pre_process_1(text) :
  ret = ""
  for i in range(len(text)):
    s = text[i]
    s = s.lower()
    t = ""
    if (s == '_' or s == '\n'):
        t += ''
    else:
        t += s 

    ret += t

  return ret

raw_text = pre_process_1(text)

# create mapping of unique characters to integers
chars = sorted(list(set(raw_text)))
#print(chars)
char_to_int = {}
ind = 0
for c in chars :
  char_to_int[c] = ind
  ind += 1


In [52]:
# Print the total characters and character vacob size
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total characters : ", n_chars)
print("Character vocab size : ", n_vocab)

Total characters :  141065
Character vocab size :  44


In [53]:
'''
Prepare dataset where the input is sequence of 100 characters and target is next character.
'''
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  # Write code here
  seq_in = raw_text[i:i+seq_length]
  seq_out = raw_text[i+seq_length]
  inp = []
  out = []
  for i in range (len(seq_in)):
    inp.append(char_to_int[seq_in[i]])

  out.append(char_to_int[seq_out])
  dataX.append(inp)
  dataY.append(out)



n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  140965


In [54]:
# reshape X to be [samples, time steps, features]
X = numpy.array(dataX)

# one hot encode the output variable
dataY = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [55]:
embedding_dim =100
max_length =100

In [56]:
model = Sequential()
model.add(keras.layers.Embedding(n_vocab, embedding_dim, input_length=max_length))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 100)          4400      
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 44)                11308     
Total params: 381,276
Trainable params: 381,276
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.fit(X, y, epochs=20, batch_size=128)

Epoch 1/20
1102/1102 [==============================] - 19s 17ms/step - loss: 2.2257
Epoch 2/20
1102/1102 [==============================] - 19s 17ms/step - loss: 1.7207
Epoch 3/20
1102/1102 [==============================] - 19s 17ms/step - loss: 1.5205
Epoch 4/20
1102/1102 [==============================] - 19s 17ms/step - loss: 1.4001
Epoch 5/20
1102/1102 [==============================] - 19s 17ms/step - loss: 1.3164
Epoch 6/20
1102/1102 [==============================] - 19s 18ms/step - loss: 1.2535
Epoch 7/20
1102/1102 [==============================] - 19s 18ms/step - loss: 1.2012
Epoch 8/20
1102/1102 [==============================] - 19s 18ms/step - loss: 1.1585
Epoch 9/20
1102/1102 [==============================] - 20s 18ms/step - loss: 1.1190
Epoch 10/20
1102/1102 [==============================] - 20s 18ms/step - loss: 1.0842
Epoch 11/20
1102/1102 [==============================] - 20s 18ms/step - loss: 1.0516
Epoch 12/20
1102/1102 [==============================] - 20s 18

In [58]:
#implement mapping of integer to character
int_to_char = {}
for key in char_to_int:
  index = char_to_int[key]
  int_to_char[index] = key


In [59]:
'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	inp = []
	for i in range(100):
		c = pattern[i]
		inp.append(char_to_int[c])
	final_string = ""
	for i in range(x):
		# Complete Code
		prediction = model.predict([inp], verbose=0)
		ind = -1
		mx = 0
		cols = prediction.shape[1]
		
		for i in range(cols):
			mx = max(mx,prediction[0][i])
	 
		for i in range(cols):
			if (prediction[0][i]==mx) :
	 			ind = i

		inp = inp[1:] + [ind]
		next_char = int_to_char[ind]		 
		final_string = final_string + next_char

	return final_string

In [60]:
# pick a random seed
random.seed(49)
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print(predict_next_100_chars(input_str,200))

 the little golden key in the window, and was going to she tried to herself “the most of the world you like to go on in the window, and the poor little thing was so much frightened that she was not a 


In [61]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.
#print(pre_process_1(input_str))
 
print(predict_next_100_chars(pre_process_1(input_str)[:100],200))

nd the moral of that is—‘the same thing as it was in the window, and the poor little thing was so larger, and the poor little thing says in the window, and was going to she tried to herself “the most 


## Character based LSTM Model 2


In [62]:
model1 = Sequential()
model1.add(keras.layers.Embedding(n_vocab, embedding_dim, input_length=max_length))
model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')

In [63]:
model1.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
2203/2203 [==============================] - 49s 22ms/step - loss: 2.1178
Epoch 2/20
2203/2203 [==============================] - 49s 22ms/step - loss: 1.5771
Epoch 3/20
2203/2203 [==============================] - 49s 22ms/step - loss: 1.3993
Epoch 4/20
2203/2203 [==============================] - 49s 22ms/step - loss: 1.2989
Epoch 5/20
2203/2203 [==============================] - 49s 22ms/step - loss: 1.2304
Epoch 6/20
2203/2203 [==============================] - 49s 22ms/step - loss: 1.1767
Epoch 7/20
2203/2203 [==============================] - 49s 22ms/step - loss: 1.1353
Epoch 8/20
2203/2203 [==============================] - 50s 23ms/step - loss: 1.0978
Epoch 9/20
2203/2203 [==============================] - 49s 22ms/step - loss: 1.0651
Epoch 10/20
2203/2203 [==============================] - 49s 22ms/step - loss: 1.0386
Epoch 11/20
2203/2203 [==============================] - 49s 22ms/step - loss: 1.0114
Epoch 12/20
2203/2203 [==============================] - 49s 22

In [64]:
# Generate the sequence similar to above methods

'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	inp = []
	for i in range(100):
			c = pattern[i]
			inp.append(char_to_int[c])
	final_string = ""
	for i in range(x):
		# Complete Code
		prediction = model.predict([inp], verbose=0)
		ind = -1
		mx = 0
		cols = prediction.shape[1]
		
		for i in range(cols):
			mx = max(mx,prediction[0][i])
	 
		for i in range(cols):
			if (prediction[0][i]==mx) :
	 			ind = i

		inp = inp[1:] + [ind]
		next_char = int_to_char[ind]		 
		final_string = final_string + next_char

	return final_string

In [65]:
# pick a random seed
random.seed(97)
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print(predict_next_100_chars(input_str,200))

trial can in the window, and the poor little thing was so much frightened that she was not a bit as she could not speak again, and she tried the trial children when they had to see the words “drink me


In [36]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.
print(pre_process_1(input_str))
 
print(predict_next_100_chars(pre_process_1(input_str)[:100],200))

the boy laughed at the fright he had caused. this time, the villagers left angrily. the third day, as the boy went up the small hill, he suddenly saw a wolf attacking his sheep. he cried as hard as he could, “wolf! wolf! wolf!”, but not  a single villager came to help him. the villagers thought that he was trying to fool them again and did not come to rescue  him or his sheep.
nd the reason is to be a little door of a sea of sight, and the white rabbit began to herself “that it might happen, you know.” alice could see the white rabbit read out a little with off the white ra


# Questions

**Question:** What are your observations based on the model(all) outputs on train data(in domain) vs unseen data(out of domain) ?

**Answer:** The output of each model is better for the training set than for the test set (unseen data). With regard to the accuracy, for each model, the accuracy on the training set increases as the epoch number increases. However, the accuracy of the models on the test set (unseen data) increases upto a maximum value, and then decreases and oscillates in values lower than the maximum. The reason for this might be overfitting. Also, for each model, the accuracy of the model on the training set is greater than the accuracy of the model on the test set.

**Question:** What did you observe in the outputs of char LSTM model1 vs char LSTM model2 ?

**Answer:**
The outputs of char-LSTM model 2 are better than the outputs for char-LSTM model1. The reason for this might be that char-LSTM model 2 has two layers whereas char-LSTM model1 has only one layer. Also , char-LSTM model2 takes more time to train than char-LSTM model1.

# Bonus (Not to be graded)

## Transformer based language model (Bert)
You can run the below cells to predict the next word using pretrained BERT model. 

In [ ]:
!pip install transformers

In [ ]:
import os
import torch
import string
from transformers import BertTokenizer, BertForMaskedLM

In [ ]:
def load_model(model_name):
  try:
    if model_name.lower() == "bert":
      bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
      return bert_tokenizer,bert_model
  except Exception as e:
    pass

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
  ignore_tokens = string.punctuation + '[PAD]'
  tokens = []
  for w in pred_idx:
    token = ''.join(tokenizer.decode(w).split())
    if token not in ignore_tokens:
      tokens.append(token.replace('##', ''))
  return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
  text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
  # if <mask> is the last token, append a "." so that models dont predict punctuation.
  if tokenizer.mask_token == text_sentence.split()[-1]:
    text_sentence += ' .'
  input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
  mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
  return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
  input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
  with torch.no_grad():
    predict = bert_model(input_ids)[0]
    print(predict.shape)
    
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
  return {'bert': bert}

In [ ]:
def get_prediction_eos(input_text):
  try:
    input_text += ' <mask>'
    res = get_all_predictions(input_text, top_clean=int(top_k))
    return res
  except Exception as error:
    pass

In [ ]:
# Below code predicts the next top_k words. You can modify this to get next n words using top_k=1 and greedy decoding it. 
top_k= 3
print('Predict next ', top_k, ' words')
model_name = 'BERT'
bert_tokenizer, bert_model  = load_model(model_name) 
input_text = "" ### GIVE YOUR INPUT STRING HERE
res = get_prediction_eos(input_text)
answer = []
print(res['bert'].split("\n"))
for i in res['bert'].split("\n"):
  answer.append(i)
  answer_as_string = "    ".join(answer)

print(answer_as_string)